In [11]:
import pandas as pd
import numpy as np
# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /Users/raissadeboer/opt/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         820 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.0

In [39]:
data = pd.read_csv('./Desktop/projects/CourseraCapstone/completedata.csv')
data = data.drop(columns = ['Unnamed: 0']) 
data = data.rename(columns = {"Neighbourhood" : "Neighborhood"}) 
data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [40]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(data['Borough'].unique()),
        data.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [41]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


## Creating a map of Toronto with the location of the different neighbourhoods

In [42]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='pink',
        fill=True,
        fill_color='#fe3523',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Define Foursquare Credentials and Version

In [43]:
CLIENT_ID = 'DVJ0NVC1TT1T5KW3SVRLEMVKZEA2CASMQMATEAJBGFK0R4NN' # your Foursquare ID
CLIENT_SECRET = 'ZKODK0QPJWCKHTPWQGA3K1TLQKCJ5RNUBI0J4HMLMUGOHMUO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DVJ0NVC1TT1T5KW3SVRLEMVKZEA2CASMQMATEAJBGFK0R4NN
CLIENT_SECRET:ZKODK0QPJWCKHTPWQGA3K1TLQKCJ5RNUBI0J4HMLMUGOHMUO


## Define a function to extract venues around a latitude and longitude

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Extract venues in the different neighborhoods

In [45]:
toronto_venues = getNearbyVenues(names=data['Neighborhood'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [46]:
print(toronto_venues.shape)
toronto_venues.head()

(1329, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


## Arrange the venues in a dataframe with onehotencoding

In [49]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
toronto_onehot.shape # There are 229 categories

(1329, 229)

## Create a frame where we get the occurrence of different categories within the different neighborhoods

In [51]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,...,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,"Woodbine Gardens,Parkview Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.0


In [52]:
toronto_grouped.shape

(100, 229)

## Get the most often occurring type of venues in each of the different neighborhoods

In [53]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                venue  freq
0          Steakhouse  0.10
1  Seafood Restaurant  0.07
2                Café  0.07
3    Asian Restaurant  0.07
4               Hotel  0.07


----Agincourt----
                       venue  freq
0                     Lounge   0.2
1  Latin American Restaurant   0.2
2               Skating Rink   0.2
3             Clothing Store   0.2
4             Breakfast Spot   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
         venue  freq
0   Playground   0.5
1         Park   0.5
2  Yoga Studio   0.0
3        Motel   0.0
4       Market   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
           venue  freq
0  Grocery Store   0.2
1    Pizza Place   0.1
2    Coffee Shop   0.1
3   Liquor Store   0.1
4     Beer Store   0.1


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place  0.22
1             Gym  0.11
2     Coffee Shop  0.11
3 

          venue  freq
0  Hockey Arena  0.33
1         Trail  0.33
2         Field  0.33
3   Yoga Studio  0.00
4         Motel  0.00


----Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips----
               venue  freq
0           Bus Line  0.25
1  Mobile Phone Shop  0.25
2     Sandwich Place  0.25
3        Pizza Place  0.25
4        Yoga Studio  0.00


----Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor----
                  venue  freq
0                   Gym  0.07
1        Discount Store  0.07
2           Social Club  0.07
3  Fast Food Restaurant  0.07
4        Sandwich Place  0.07


----L'Amoreaux West----
                  venue  freq
0    Chinese Restaurant  0.18
1  Fast Food Restaurant  0.18
2           Pizza Place  0.09
3           Coffee Shop  0.09
4        Breakfast Spot  0.09


----Lawrence Heights,Lawrence Manor----
                    venue  freq
0          Clothing Store  0.27
1  Furniture / Home Store  0.13
2 

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Put this into a pandas dataframe

In [131]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Asian Restaurant,Seafood Restaurant,Café,Hotel,Concert Hall,Lounge,Smoke Shop,Speakeasy,Plaza
1,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Clothing Store,Diner,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Women's Store,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,American Restaurant
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Skating Rink,Pool,Pub,Sandwich Place,Coffee Shop,Gym,Gas Station,Cuban Restaurant


# Create clusters of the different neighborhoods

In [132]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 0, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

## A dataframe with the 10 most occurring venues for different neighborhoods in Toronto

In [133]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

toronto_merged = data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged = toronto_merged[pd.notnull(toronto_merged['ClusterLabels'])]
toronto_merged['ClusterLabels'] = toronto_merged['ClusterLabels'].astype(int)

toronto_merged.head(10) 


,Postcode,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Women's Store,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4,Bar,Women's Store,Diner,Farmers Market,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,4,Rental Car Location,Spa,Breakfast Spot,Pizza Place,Electronics Store,Intersection,Medical Center,Mexican Restaurant,Dim Sum Restaurant,Ethiopian Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Coffee Shop,Korean Restaurant,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Bank,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Gas Station,Dog Run,Diner
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Playground,Women's Store,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,4,Coffee Shop,Convenience Store,Bus Station,Department Store,Discount Store,Women's Store,Diner,Falafel Restaurant,Fabric Shop,Event Space
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,4,Bakery,Park,Bus Line,Bus Station,Metro Station,Ice Cream Shop,Soccer Field,Discount Store,Fabric Shop,Event Space
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,4,Motel,American Restaurant,Dessert Shop,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,4,Café,General Entertainment,Skating Rink,College Stadium,Construction & Landscaping,Convenience Store,Fabric Shop,Event Space,Ethiopian Restaurant,Comic Shop


# Create a map with the different neighborhoods and how they cluster together

In [135]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Explore the different neighborhood clusters

### Cluster 1
#### Seems that neighborhoods in this cluser have a lot of parks

In [136]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0,Park,Playground,Women's Store,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
20,"Silver Hills,York Mills",0,Cafeteria,Park,Women's Store,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
23,York Mills West,0,Park,Convenience Store,Bank,Women's Store,Diner,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store
30,"CFB Toronto,Downsview East",0,Park,Airport,Snack Place,Women's Store,Dim Sum Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
31,Downsview West,0,Grocery Store,Park,Shopping Mall,Bank,Hotel,Dim Sum Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store
32,Downsview Central,0,Baseball Field,Food Truck,Home Service,Women's Store,Diner,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store
37,The Beaches,0,Health Food Store,Trail,Park,Pub,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Women's Store
40,East Toronto,0,Coffee Shop,Park,Convenience Store,Women's Store,Dim Sum Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
44,Lawrence Park,0,Photography Studio,Park,Bus Line,Swim School,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
48,"Moore Park,Summerhill East",0,Tennis Court,Park,Department Store,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore


### Cluster 2
#### This cluster only consists of 1 neighborhood and has a playground as the most common venue

In [138]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,1,Playground,Women's Store,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore


### Cluster 3
#### This cluster of neighborhoods consists of a lot of restaurants and especially fastfood restaurants

In [139]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",2,Fast Food Restaurant,Women's Store,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
25,Parkwoods,2,Fast Food Restaurant,Park,Food & Drink Shop,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
80,"Del Ray,Keelesdale,Mount Dennis,Silverthorn",2,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Restaurant,Women's Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Cluster 4
#### This is also a cluster of only 1 neighborhood. The most common venue is agarden, and there are some stores there.

In [140]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Roselawn,3,Garden,Women's Store,Dessert Shop,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Cluster 5
#### This cluster has a lot of different types of restautrants and stores

In [141]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek,Rouge Hill,Port Union",4,Bar,Women's Store,Diner,Farmers Market,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
2,"Guildwood,Morningside,West Hill",4,Rental Car Location,Spa,Breakfast Spot,Pizza Place,Electronics Store,Intersection,Medical Center,Mexican Restaurant,Dim Sum Restaurant,Ethiopian Restaurant
3,Woburn,4,Coffee Shop,Korean Restaurant,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,Cedarbrae,4,Bank,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Gas Station,Dog Run,Diner
6,"East Birchmount Park,Ionview,Kennedy Park",4,Coffee Shop,Convenience Store,Bus Station,Department Store,Discount Store,Women's Store,Diner,Falafel Restaurant,Fabric Shop,Event Space
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Humber Summit,4,Pizza Place,Shopping Mall,Women's Store,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
99,Westmount,4,Pizza Place,Coffee Shop,Chinese Restaurant,Sandwich Place,Intersection,Discount Store,Women's Store,Dim Sum Restaurant,Event Space,Ethiopian Restaurant
100,"Kingsview Village,Martin Grove Gardens,Richvie...",4,Pizza Place,Bus Line,Sandwich Place,Mobile Phone Shop,Women's Store,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
101,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",4,Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,American Restaurant
